# Exampel of Linear Regression with different methods

##       - We will look at how to prepare your data
##       - How to create a Linear Regression model with sklearn in Pandas
  - What is a model
  - How evaluate it's performance
  -Some insights in alternative methods


[Training data from Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview   )


---
</br>  


In [25]:
# först behöver vi python-biblioteket pandas och kommer att kalla det pd
# samt numpy-bibliotetket numpy och vi kallar det np
import numpy as np
import pandas as pd

# nu laddar vi in båda csv-filerna 
# De kommer från kaggle

urlTrain="https://raw.githubusercontent.com/carokanns/ML/master/train.csv"
urlTest = "https://raw.githubusercontent.com/carokanns/ML/master/test.csv"

# Här skapar vi två DataFrame"
hus=pd.read_csv(urlTrain)   # Our house data  
KaggleTest=pd.read_csv(urlTest)  # Kaggle test (that we will not use more than to look at)
KaggleTest.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


Note that there is no SalePrice in Ktest. This is the file that Kaggle want you to run and upload in order to participate. 

# Testar Pipelines här

In [26]:
hus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [27]:
y= hus['SalePrice']
X= hus.drop(['SalePrice','Id'],axis=1)
    

In [28]:
# Använd catBoost för att bestämma feature_importance för att veta ilka kolumner som är bäst
from sklearn.impute import SimpleImputer
from catboost import CatBoostRegressor,Pool
from sklearn.model_selection import train_test_split

cat_features = X.select_dtypes(include='O').columns.tolist()
imp = SimpleImputer(strategy='constant',fill_value='aNAN')

cats = pd.DataFrame(imp.fit_transform(X[cat_features]),columns=cat_features)
nums = X.drop(cat_features,axis=1)
nums.shape
Xt=pd.concat([cats,nums],axis=1)

X_train, X_test, y_train, y_test = train_test_split(Xt, y, random_state=3, test_size=0.20)

train_pool=Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(data=X_test.copy(), label=y_test, cat_features=cat_features)
cr = CatBoostRegressor()

cr.fit(train_pool,eval_set=test_pool,
        use_best_model=True,
        early_stopping_rounds=200,
        verbose=100)


Learning rate set to 0.04745
0:	learn: 78943.8242762	test: 67829.0251166	best: 67829.0251166 (0)	total: 73.3ms	remaining: 1m 13s
100:	learn: 22403.1461677	test: 26398.3953761	best: 26388.4719041 (97)	total: 10.2s	remaining: 1m 31s
200:	learn: 16824.4835166	test: 25464.2691616	best: 25464.2691616 (200)	total: 20.6s	remaining: 1m 21s
300:	learn: 14179.0761421	test: 25203.3822573	best: 25203.3822573 (300)	total: 30.9s	remaining: 1m 11s
400:	learn: 12336.2011627	test: 25070.7700106	best: 25070.7700106 (400)	total: 41.3s	remaining: 1m 1s
500:	learn: 11098.4513616	test: 25143.3778367	best: 25058.4870571 (456)	total: 51.9s	remaining: 51.7s
600:	learn: 10082.4880817	test: 25270.0367995	best: 25058.4870571 (456)	total: 1m 2s	remaining: 41.4s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 25058.48706
bestIteration = 456

Shrink model to first 457 iterations.


In [29]:
cr.get_feature_importance(prettified=True).head(25)

,Feature Id,Importances
0,OverallQual,21.205186
1,GrLivArea,15.250622
2,1stFlrSF,6.134727
3,TotalBsmtSF,4.618628
4,GarageCars,4.272173
5,BsmtQual,4.076550
6,BsmtFinSF1,3.968200
7,LotArea,2.880302
8,2ndFlrSF,2.836166
9,FireplaceQu,2.658106


In [30]:
# selektera de 10 "bästa" (feature_importance)
selected_features=cr.get_feature_importance(prettified=True).iloc[:10,0].values.tolist()
selected_features += ['GarageYrBlt']   # one numeric with missing values
print(selected_features)


['OverallQual', 'GrLivArea', '1stFlrSF', 'TotalBsmtSF', 'GarageCars', 'BsmtQual', 'BsmtFinSF1', 'LotArea', '2ndFlrSF', 'FireplaceQu', 'GarageYrBlt']


# Nu har vi valt 11 st kolumner att labba med

In [31]:
# Börja om från början med originalet
y= hus['SalePrice']
X= hus[selected_features]

X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OverallQual  1460 non-null   int64  
 1   GrLivArea    1460 non-null   int64  
 2   1stFlrSF     1460 non-null   int64  
 3   TotalBsmtSF  1460 non-null   int64  
 4   GarageCars   1460 non-null   int64  
 5   BsmtQual     1423 non-null   object 
 6   BsmtFinSF1   1460 non-null   int64  
 7   LotArea      1460 non-null   int64  
 8   2ndFlrSF     1460 non-null   int64  
 9   FireplaceQu  770 non-null    object 
 10  GarageYrBlt  1379 non-null   float64
dtypes: float64(1), int64(8), object(2)
memory usage: 125.6+ KB


In [32]:
# hantera nans i icke numeriska (categorical) features 'BsmtQual' och 'FireplaceQu'
#### BsmtQual ####
# Ex	Excellent (100+ inches)	
# Gd	Good (90-99 inches)
# TA	Typical (80-89 inches)
# Fa	Fair (70-79 inches)
# Po	Poor (<70 inches
# NA	No Basement

print(X.BsmtQual.value_counts())
print(f'NANs: {X.BsmtQual.isna().sum()}')

imp = SimpleImputer(strategy='most_frequent')
X.BsmtQual=imp.fit_transform(X[['BsmtQual']])

print(f'NANs: {X.BsmtQual.isna().sum()}')


TA    649
Gd    618
Ex    121
Fa     35
Name: BsmtQual, dtype: int64
NANs: 37
NANs: 0


In [33]:
# hantera nans i icke numeriska (categorical) features 'BsmtQual' och 'FireplaceQu'
#### FireplaceQu ####
# Ex	Excellent - Exceptional Masonry Fireplace
# Gd	Good - Masonry Fireplace in main level
# TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
# Fa	Fair - Prefabricated Fireplace in basement
# Po	Poor - Ben Franklin Stove
# NA	No FireplaceX.BsmtQual.value_counts()
print(X.FireplaceQu.value_counts())
print(f'NANs: {X.FireplaceQu.isna().sum()}')

imp = SimpleImputer(strategy='constant', fill_value="TA") # För omväxlings skull
X.FireplaceQu=imp.fit_transform(X[['FireplaceQu']])

print(f'NANs: {X.FireplaceQu.isna().sum()}')


Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64
NANs: 690
NANs: 0


In [34]:
X[X.GarageYrBlt.isna()].shape[0] == X[X.GarageCars==0].shape[0]

### Vi kan förutsätta att NaN i GarageYrBlt betyder att Garage saknas 
### Antagligen kan vi sätta GarageYrBlt bättre än CatBoost, genom att sätta till 0

print("Första testet: CatBoost får själv hanter GarageYrBlt Nan")
print(f'NANs: {X.GarageYrBlt.isna().sum()}')


Första testet: CatBoost får själv hanter GarageYrBlt Nan
NANs: 81


In [35]:
X.isna().sum()
# Inga Nan kvar

OverallQual     0
GrLivArea       0
1stFlrSF        0
TotalBsmtSF     0
GarageCars      0
BsmtQual        0
BsmtFinSF1      0
LotArea         0
2ndFlrSF        0
FireplaceQu     0
GarageYrBlt    81
dtype: int64

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.20)

train_pool=Pool(data=X_train, label=y_train, cat_features=['BsmtQual', 'FireplaceQu'])
test_pool = Pool(data=X_test.copy(), label=y_test, cat_features=['BsmtQual', 'FireplaceQu'])
cr = CatBoostRegressor()

cr.fit(train_pool,eval_set=test_pool,
        use_best_model=True,
        early_stopping_rounds=200,
        verbose=100)


Learning rate set to 0.04745
0:	learn: 79148.1289977	test: 68202.5324940	best: 68202.5324940 (0)	total: 73.7ms	remaining: 1m 13s
100:	learn: 23396.4313627	test: 27698.5424084	best: 27698.5424084 (100)	total: 7.07s	remaining: 1m 2s
200:	learn: 19784.4772263	test: 26899.0584221	best: 26879.6153149 (198)	total: 14s	remaining: 55.6s
300:	learn: 17321.9434111	test: 26368.6892920	best: 26351.2067018 (298)	total: 21.1s	remaining: 49.1s
400:	learn: 15363.4448253	test: 26137.2085404	best: 26111.2634259 (353)	total: 28.3s	remaining: 42.3s
500:	learn: 14061.2821697	test: 26118.0744727	best: 26111.2634259 (353)	total: 35.5s	remaining: 35.3s
600:	learn: 12978.5537146	test: 26230.6631609	best: 26091.6744985 (525)	total: 42.6s	remaining: 28.3s
700:	learn: 12077.9161572	test: 26210.9895108	best: 26091.6744985 (525)	total: 49.6s	remaining: 21.1s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 26091.6745
bestIteration = 525

Shrink model to first 526 iterations.


In [37]:
cr.get_feature_importance(prettified=True)

,Feature Id,Importances
0,OverallQual,24.561414
1,GrLivArea,15.031766
2,2ndFlrSF,10.966465
3,TotalBsmtSF,9.839796
4,BsmtQual,7.648170
5,BsmtFinSF1,7.235856
6,LotArea,6.731918
7,1stFlrSF,6.687291
8,GarageYrBlt,6.050036
9,GarageCars,4.104483


## Evaluate första testet

In [13]:
from sklearn import metrics

MAE=metrics.median_absolute_error(y_test,cr.predict(test_pool))
print(MAE)
y_test.describe()

11901.07709283859


count       292.000000
mean     178248.863014
std       70389.950788
min       55000.000000
25%      128800.000000
50%      158500.000000
75%      202600.000000
max      466500.000000
Name: SalePrice, dtype: float64

### Andra testet

In [14]:

print("Andra testet: Vi hanterar själva GarageYrBlt Nan")
X2=X.copy()
y2=y
print(f'NANs: {X2.GarageYrBlt.isna().sum()}')

imp2 = SimpleImputer(strategy='constant', fill_value=0) # Bättre än CatBoost!?
X2.GarageYrBlt=imp2.fit_transform(X2[['GarageYrBlt']])

print(f'NANs: {X2.GarageYrBlt.isna().sum()}')


Andra testet: Vi hanterar själva GarageYrBlt Nan
NANs: 81
NANs: 0


In [15]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=3, test_size=0.20)

train2_pool=Pool(data=X2_train.copy(), label=y2_train, cat_features=['BsmtQual', 'FireplaceQu'])
test2_pool = Pool(data=X2_test.copy(), label=y2_test, cat_features=['BsmtQual', 'FireplaceQu'])
cr2 = CatBoostRegressor()

cr2.fit(train2_pool,eval_set=test2_pool,
        use_best_model=True,
        early_stopping_rounds=200,
        verbose=100)


Learning rate set to 0.04745
0:	learn: 79148.1289977	test: 68202.5324940	best: 68202.5324940 (0)	total: 60.3ms	remaining: 1m
100:	learn: 23396.4313627	test: 27698.5424084	best: 27698.5424084 (100)	total: 6.88s	remaining: 1m 1s
200:	learn: 19784.4772263	test: 26899.0584221	best: 26879.6153149 (198)	total: 13.5s	remaining: 53.7s
300:	learn: 17321.9434111	test: 26368.6892920	best: 26351.2067018 (298)	total: 20.7s	remaining: 48s
400:	learn: 15363.4448253	test: 26137.2085404	best: 26111.2634259 (353)	total: 27.8s	remaining: 41.6s
500:	learn: 14061.2821697	test: 26118.0744727	best: 26111.2634259 (353)	total: 35s	remaining: 34.9s
600:	learn: 12978.5537146	test: 26230.6631609	best: 26091.6744985 (525)	total: 42s	remaining: 27.9s
700:	learn: 12077.9161572	test: 26210.9895108	best: 26091.6744985 (525)	total: 49.1s	remaining: 20.9s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 26091.6745
bestIteration = 525

Shrink model to first 526 iterations.


## Evaluate andra testet 
   Blir exakt samma!

In [16]:
print(y2_test.describe())
MAE=metrics.median_absolute_error(y2_test,cr2.predict(test2_pool))
print(f'\nMAE: {MAE}')


count       292.000000
mean     178248.863014
std       70389.950788
min       55000.000000
25%      128800.000000
50%      158500.000000
75%      202600.000000
max      466500.000000
Name: SalePrice, dtype: float64

MAE: 11901.07709283859


In [22]:
cr2.get_feature_importance(prettified=True)

,Feature Id,Importances
0,OverallQual,24.561414
1,GrLivArea,15.031766
2,2ndFlrSF,10.966465
3,TotalBsmtSF,9.839796
4,BsmtQual,7.648170
5,BsmtFinSF1,7.235856
6,LotArea,6.731918
7,1stFlrSF,6.687291
8,GarageYrBlt,6.050036
9,GarageCars,4.104483


[Remember the list of useful pandas commands](https://colab.research.google.com/drive/1_4ib8P-DQt8zjb0iY7pZhqUz4JpuiTwt#scrollTo=sFNHlHUm8Qev&line=9&uniqifier=1)


# The same thing with pipelines

In [17]:
# Börja om från början med originalet
y= hus['SalePrice']
X= hus[selected_features]

X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OverallQual  1460 non-null   int64  
 1   GrLivArea    1460 non-null   int64  
 2   1stFlrSF     1460 non-null   int64  
 3   TotalBsmtSF  1460 non-null   int64  
 4   GarageCars   1460 non-null   int64  
 5   BsmtQual     1423 non-null   object 
 6   BsmtFinSF1   1460 non-null   int64  
 7   LotArea      1460 non-null   int64  
 8   2ndFlrSF     1460 non-null   int64  
 9   FireplaceQu  770 non-null    object 
 10  GarageYrBlt  1379 non-null   float64
dtypes: float64(1), int64(8), object(2)
memory usage: 125.6+ KB


In [18]:
cat_features = X.columns[X.dtypes=='object']

In [19]:
# version ett - låt catboost bestämma GarageYrBlt föt NaNs
train2_pool=Pool(data=X2_train, label=y2_train, cat_features=['BsmtQual', 'FireplaceQu'])
test2_pool = Pool(data=X2_test.copy(), label=y2_test, cat_features=['BsmtQual', 'FireplaceQu'])

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import  make_column_transformer
col_trans = make_column_transformer (
    (SimpleImputer(strategy='constant', fill_value="TA"),['FireplaceQu']),
    (SimpleImputer(strategy='most_frequent'),['BsmtQual']),
    remainder='passthrough'
)
col_trans.fit_transform(X2_train)


array([['TA', 'Gd', 7.0, ..., 7314.0, 0.0, 2007.0],
       ['TA', 'TA', 6.0, ..., 11888.0, 689.0, 1930.0],
       ['TA', 'Gd', 7.0, ..., 8450.0, 0.0, 2003.0],
       ...,
       ['TA', 'TA', 3.0, ..., 5925.0, 368.0, 0.0],
       ['TA', 'Gd', 5.0, ..., 7200.0, 0.0, 1974.0],
       ['Gd', 'TA', 6.0, ..., 11512.0, 0.0, 1959.0]], dtype=object)

In [20]:
cr = CatBoostRegressor(use_best_model=True,cat_features=cat_features,early_stopping_rounds=200,verbose=100)
# col_trans.fit_transform(X)
pipe1 = Pipeline(steps=[('cols', col_trans),
                        ('est',cr )])

pipe1.fit(X2_train,y2_train,est__eval_set=test2_pool)


CatBoostError: features parameter contains string value 'BsmtQual' but feature names for a dataset are not specified

In [ ]:
# A short recap to get information from a DataFrame
# Let's do some typical pandas commands
hus.shape      # (how many rows and columns)
hus.dtypes # what types of features (columns) do we have
hus["Alley"].unique() # Check out unique values for one feature
hus[hus.Alley.notna()].Alley.count() # How many rows with not missing data
hus.info()         # info per feature about type and number of "not missing" values
hus.describe()     #  summary of all numerical columns

In [ ]:
hus[['LotArea','Id']] # avoid this - use loc or iloc
hus.loc[0:5,'LotArea':'Alley'] 
#hus.loc[5:10,['LotArea','Id','Street']] 
hus.columns # returns a NumPy array

In [ ]:
hus.describe()

# Feature Enginering
**We will:**
- **Take care of Missing Values** (sometimes called nulls, NaN, NA etc. The data type is np.NaN)
- **Set numerical values on categoricals** (all data **must** be numeric in order to use in ML). A categorical is typically:
    - **ordinal** - ordered even if not numeric
    - **"binary"** - exactly two values, either one or the other
    - **codes, strings, abbreviations** etc without any internal ordering
    
- In a real case we should be much more aquainted with the data and go through each feature separately. It's meaning, its' distribution and more. This process would be best with a contact with the estate agents 
- we will continue to fix some features, enough to make it possible to use for sklearn


In [ ]:
import matplotlib.pyplot as plt
#plt.style='ggplot'
#plt.rcParams['figure.figsize'] = (10,6)
import seaborn as sns

hus.SalePrice.describe()

In [ ]:
print(hus.SalePrice.skew())
plt.hist(hus.SalePrice)
plt.show()

*The SalePrice distribution is skew to the left*  
We can transform the column with np.log()

***NB. We have to transform it back later on with np.exp() in order to see the real prices***

In [ ]:
#What about GareageArea vs GarageYrBlt?
tf=hus.GarageArea==0
hus.loc[tf].GarageYrBlt  # GarageYrBlt in all rows with GarageArea==0


In [ ]:
logPrice = np.log(hus.SalePrice)
print(logPrice.skew())
plt.hist(logPrice)
plt.show()

In [ ]:
husMod =hus.copy()
#
husMod['SalePrice'] = np.log(hus.SalePrice)
# From now on we will use husMod and keep the original in hus

In [ ]:
numFeatures = hus.select_dtypes(include=[np.number])
nfCorr = numFeatures.corr()  # how are the features correlated?
nfCorr.head()
# 0 is no correlation at all, positive or negative correlations are just opposite directions
print(nfCorr['SalePrice'].sort_values(ascending=False).head()) # positive correlations 
nfCorr['SalePrice'].sort_values(ascending=False).tail() # negative correlations


In [ ]:
nfCorr['SalePrice'].sort_values(ascending=False)[1:10].index
corrMax=nfCorr['SalePrice'].sort_values(ascending=False)[1:10].index
husMod.loc[:,corrMax[0:9]]
plt.subplots(figsize=(20, 15))
sns.heatmap(husMod.loc[:,corrMax[0:9]].corr(), annot=True)

### We could also check the least correlated to SalePrice in similar ways but...   
We can't rely on only one to one correlations to SalePrice. There might be combinations so PCA is a better way to find out which features we might try to take out in order to reduse noice


---

Now let's find if there are some outliers. Here we will check just a few features, there could be more...

In [ ]:
husMod.GarageArea.sort_values()
# we can see a number of zeroes, meaning that there is no garage. Let's forget them for a moment
husMod.GarageArea[husMod.GarageArea>0].sort_values()
# There are big diffferenses between the smallest and the largest garages
# What can a plot show us?


In [ ]:
plt.rcParams['figure.figsize'] = (15,5)
plt.scatter(x=husMod['GarageArea'], y=husMod.SalePrice)
#There are some really bad outliers - somewhere above 1100 
temp = husMod[husMod['GarageArea']<1200]
plt.scatter(x=temp['GarageArea'], y=temp.SalePrice)  # Kommer att överlappa föregående graf
# De överlappar varandra med olika färger
#husMod=temp
#husMod.shape



# Take care of Missing Values
A missing value is displayed as NaN and it is the NumPy type np.NaN. We can't feed missing values into our algorthms
- Go back to the source (Maybe there are values available or the estate agency has better estimations than we can get from our automatic approach)
- Remove the entire feature (column)
- Remove rows
- Use fake values:
    - **Mean**    (Numerical features)
    - **Median**  (Numerical features 
    - **Mode** = most frequent value. (for categorical feautures, not normally for numerical)
    

In [ ]:
husMod.isna()  # NaN true or false
husMod.isna().sum()  # sum all the NaN (True) per feature
husMod.isna().sum().sort_values(ascending=False) # sorted by number of NaN's
husMod.columns[husMod.isna().sum()>0]  # These are all the features with NaN
husMod.columns[husMod.isna().sum()>0].shape  # how many
sum(husMod.isnull().sum()>0)  # how many (alternative way)
husMod.isna().sum().sort_values(ascending=False).head(20)  # we got 19 above
 

In [ ]:
# The first 5 columms we just better remove - too many missing values

print("Before:", husMod.shape)
husMod=husMod.drop(['PoolQC','Fence', 'MiscFeature', 'Alley', 'FireplaceQu'], axis=1)
print("After: ",husMod.shape)
print(husMod.isnull().sum())
sum(husMod.isnull().sum() >250)  # Många NaN


In [ ]:
nullV = pd.DataFrame(husMod.isnull().sum().sort_values(ascending=False))
print(nullV.shape)
nullV[:20]

Only LotFrontage has >250 NaN's. Let's keep it and the others. We will handle them later on

# Take care of Categorical Features
## We will transform them to numericals
- ordinal - Possible to set numeric values in the same order
- others - One-hot encoding

In [ ]:
cat = husMod.select_dtypes(exclude=[np.number]) # select all categorical features
print(cat.shape)
cat.describe() # check the count of non-nulls

# One-hot encoding

start with Street that have two unique values

In [ ]:
print("Before:")
print(husMod.Street.dtype)
husMod.Street.value_counts()

In [ ]:
husMod['enc_street']=pd.get_dummies(husMod['Street'], prefix='Street', drop_first=True) # create enc_street with 1 and 0
husMod.loc[50:55,'enc_street']  # I happen to know that row 52 is one of the Grvl values :-)  - Now it is translated to 0
print("After:")
husMod.enc_street.value_counts()

# Question:

There are now 6 enc_street==0 (that is Street==Grvl)  
Find out exactly where they are!

Try pandas referens https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html 


In [ ]:
husMod[husMod['enc_street']==0]                         # Indexing and selecting data / Boolean indexing
husMod.loc[lambda husMod: husMod.enc_street == 0, :]   # Indexing and selecting data / Selection by callable
husMod.enc_street.sort_values(ascending=True).head(10)          # sorting

# Question:
We have now two binary columns for Street  
What to do with the Street column?


In [ ]:
husMod.drop('Street', axis=1, inplace=True)


In [ ]:
cat.shape  # Vad kan vi göra?
# Leta upp alla categoricals
# Leta upp ordinal och ev sätt nummer-sekvens alt get-dummies
# kör get_dummies (one hot encoding)
# Kör get-dummies på dem

# Example of more than 2 values
print(cat.MSZoning.value_counts(),"\n")
h=husMod

print("h.shape before",h.shape,"\n")

dum = pd.get_dummies(h['MSZoning'], prefix='MSZ', drop_first=True)
print(dum,"\n")
h=pd.concat([h,dum],axis=1).drop('MSZoning',axis=1)  #concatenate and drop

print("h.shape after",h.shape)

# Nu definierar jag en funktion som kan köra igenom och fixa alla Categoricals

In [ ]:
def cat_onehot(columns):
    hM = husMod.copy()
    h = husMod
    i=0
    for field in columns:
        print(field)
        dum=pd.get_dummies(hM[field],prefix="enc",drop_first=True)
        hM.drop([field], axis=1,inplace=True)
        if i==0:
            h = dum.copy()
        else:
            h = pd.concat([h,dum],axis=1)
        
        i=i+1
        
    h=pd.concat([h,hM],axis=1)
    return h

In [ ]:
cat = husMod.select_dtypes(exclude=[np.number])
husMod=cat_onehot(cat.columns)
## No need for: husMod.drop(cat.columns,axis=1,inplace=True)
husMod

In [ ]:
# ta bort dubletter
husMod=husMod.loc[:,~husMod.columns.duplicated()]
#Kolumner som är exakt lika ger ingen extra information till sklearn
husMod.shape

# We still have missing values to take care of

In [ ]:
husMod.isna().sum().sort_values(ascending=False).head(20)  # we had 19 before

In [ ]:
husMod.select_dtypes(exclude=[np.number]).columns.shape  # No categoricals left
print("Before: ", sum(husMod.isnull().sum() != 0))

husMod=husMod.fillna(husMod.mean())        # Ta bort alla numeriska
print("after mean: ", sum(husMod.isnull().sum() != 0))
husMod=husMod.fillna(husMod.mode())
print("after mode: ", sum(husMod.isnull().sum() != 0))

# we know that the only features left with NaN are categorical
for column in husMod.columns:
    husMod[column].fillna(husMod[column].mode()[0], inplace=True)
    
print("After all:", sum(husMod.isnull().sum() != 0))

cat.columns
husMod.head(19)




# Create Polynomial features
You can adapt a feature to a polynomial distribution  
We will not do this here but this is an example

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2) # degree of polynomial
poly.fit(hus[['LotArea']])
polyLot = poly.transform(husMod[['LotArea']])
print(polyLot.shape)
print(husMod.shape)
# We would then replace LotArea with these 3 columns
pd.concat([pd.DataFrame(polyLot),husMod],axis=1,join='inner').drop('LotArea',axis=1)





---

</br>  
# Select model/method
Det finns massor med olika typer av ML-metoder och tekniker att välja från
Vi skall titta på 
- classic linear regression
- XGBoost - creates trees
- Neurala Networks


# Build the model

We will use SalePrice as Y-values and the rest of husMod as X-values. That's what sklearn wants.

'Id' is not useful or learning


In [ ]:
y = husMod.SalePrice      # np.log()SalePrice!
X = husMod.drop(['SalePrice', 'Id'], axis=1)
print(y.shape,"\n",X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.30)

# sklearn has created a pair för learning (X_train och Y_train) 
# and a pair (30% of husMod) for validation (y_train och y_test)

# Question:
Why do we need both train and test?

# Create an instans of LinearRegression


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

linr = LinearRegression()
# now run the actual learning with "fit"
m=linr.fit(X_train, y_train)
m

# How to meassure the result?



![R2](https://drive.google.com/uc?id=1KRm3mPCro1QIyHl8piDiijOd78z676YY)

In [ ]:
m.score(X_test, y_test) # Ett enkelt sätt att mäta - 0 = alla fel, 1 = alla rätt

# Det finns många varianter av mätning
## Den praktiska skillnaden är hur stor påverkan outliers har på felet  

### Formel för MSE (Mean Square Error): $$ MSE = \frac{1}{N}\sum_{i=1}^{N}{(y_i-\hat{y}_i)^2} $$

### Formel för MAE (Mean Absolute Error): $$ MAE = \frac{1}{N}\sum_{i=1}^{N}{|y_i-\hat{y}_i|} $$
 
### Formel för MAE (Root Mean Square Error): $$ RMSE = \sqrt{MSE(y_i, \hat{y}_i))} $$

---
### Kaggle Boston huspriser: Root Mean Square Log Error 
### Formel för RMSLE: $$ RMSLE = \sqrt{MSE(log(y_i + 1), log(\hat{y}_i + 1))} $$  




In [ ]:
# En Python-funktion för att enkelt kunna ge både R2 och RMSE för både train och test
# Kaggel want RMSE on log-value of the SalePrice
# np.log is already done on SalePrice so it is no need to log again for rmse of log-values

def score(model):

    model.fit(X_train, y_train)
    
    for X, y in ((X_train, y_train), (X_test, y_test)):
        y_predict = model.predict(X)
        rmse = np.sqrt(np.mean((y-y_predict)**2))
        r2 = r2_score( np.exp(y), np.exp(y_predict))
    
        if hasattr(model, 'oob_score_'):
            print("rmse:",rmse, "r2:",r2, "oob",m.oob_score_)
        else:
            print("rmse:",rmse, "r2:",r2)

In [ ]:

# r2 mäter hur nära vår prediction är linear regression linjen/planet
# RMSE är vad kaggle ofta mäter. Root Mean Square Error
score(m)



# RandomForest Regressor
Ett alternativ till klassisk Linear Regression. 
Bygger n st träd och väljer en viktning mellan träden

In [ ]:
n=150
from sklearn.ensemble import RandomForestRegressor
m = RandomForestRegressor(n_estimators=n, n_jobs=-1)
print(m)
print("\n")
score(m)

In [ ]:
np.exp(y_test).head()  # Om vi vill se SalePrice i korrekt form

# Artificial Neural Network
Ytterligare ett av många alternativ till Linear Regression
1. Vi måste normalisera all data till [0,1] eller [-1 - +1]
2. Vi behöver bestämma antal hidden layers och antal noder i varje hidden layer
3. Vi behöver bestämma en fasligt massa andra parametrar - ***HyperParameters***

# 1. Normalizing Data

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()  
# fit bara på train data
scaler.fit(X_train)  

In [ ]:
XB_train = X_train
XB_test = X_test
XB_train = scaler.transform(XB_train)  
# apply same transformation to test data
XB_test = scaler.transform(XB_test)
#print('XB_test', XB_test.shape)
XB_train.shape
XB_train


In [ ]:
print(y_train.describe())
yB_train = (y_train-y_train.min())/(y_train.max()-y_train.min())
yB_test =  (y_test-y_test.min())/(y_test.max()-y_test.min())
yB_train = yB_train.values
yB_test = yB_test.values

In [ ]:
from sklearn.neural_network import MLPRegressor   # Vi väljer ett Neuralt Nät av typen MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing


In [ ]:
# Här definierar vi en Python-funktion för att enklare kunna köra vårt Neaurala Nät  
# Vi väljer lika många noder i hidden layer som noder i input-datat (X_train)
def run_ANN(X_train, y_train, X_test, y_test):
    # Test on the boston dataset
    mlp = MLPRegressor(hidden_layer_sizes=X_train.shape[0],
                          max_iter=200, random_state=1,
                          activation='relu')
    mlp.fit(X_train, y_train)
    trainScore=mlp.score(X_train, y_train)

    y=mlp.fit(X_test, y_test)
    testScore = mlp.score(X_test, y_test)
    print("Train R2:",trainScore) 
    print("Test R2 :",testScore)
    

In [ ]:
run_ANN(XB_train, y_train.values, XB_test, y_test.values)
